#### Overview
* 공간 데이터간 거리 계산
* 예) 특정 공간과 가장 가까운 다른 공간 검색(행사장과 가장 가까운 전철역 등)

#### Import Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# scipy: 과학 및 엔지니어링 분야 기능 제공 라이브러
# KDTree: K-차원트리로 가장 가까운 이웃을 검색하며, cKDTree는 c로 구현되 더 빠름
from scipy.spatial import cKDTree, KDTree
# shapely: 공간데이터 처리 라이브러리, 공간데이터의 형상(shape)과 관계를 다루는데 사용
# Point: 관계를 표현하기 위해 각 노드를 정의할때 사용(2,3차원으로 지점 표시)
from shapely.geometry import Point

#### Fetching Data

##### Festival Data

In [4]:
FESTIVAL_SHP_PATH = os.path.join(os.getcwd(), '89_data', 'shp', 'festival_pt.shp')
festival_gdf = gpd.read_file(FESTIVAL_SHP_PATH)

In [5]:
festival_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   name        47 non-null     object  
 1   organizati  47 non-null     object  
 2   geometry    47 non-null     geometry
dtypes: geometry(1), object(2)
memory usage: 1.2+ KB


In [6]:
festival_gdf.head()

,name,organizati,geometry
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031)
1,코로나로 인해 전체 일정 취소,서울특별시 강서구,POINT (942927.290 1952222.192)
2,강동선사문화축제,서울특별시 강동구,POINT (967395.183 1951230.727)
3,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (965465.452 1949694.336)
4,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (964472.438 1949878.657)


##### Subway Data

In [7]:
SUBWAY_SHP_PATH = os.path.join(os.getcwd(), '89_data', 'shp', 'subway_pt.shp')
subway_gdf = gpd.read_file(SUBWAY_SHP_PATH)
subway_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 385 entries, 0 to 384
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   LINE_NM     385 non-null    object  
 1   YRMN        381 non-null    object  
 2   STATION_NM  385 non-null    object  
 3   IN_CNT      385 non-null    float64 
 4   OUT_CNT     385 non-null    float64 
 5   IN_OUT      385 non-null    float64 
 6   IN_OUT_SCA  385 non-null    float64 
 7   geometry    385 non-null    geometry
dtypes: float64(4), geometry(1), object(3)
memory usage: 24.2+ KB


In [8]:
subway_gdf.head()

,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,geometry
0,우이신설선,202112,신설동,55769.0,54272.0,110041.0,0.024565,POINT (957902.000 1953075.366)
1,우이신설선,202112,보문,41012.0,41211.0,82223.0,0.018187,POINT (957566.287 1954096.800)
2,우이신설선,202112,성신여대입구(돈암),91571.0,101326.0,192897.0,0.043562,POINT (957317.436 1954894.797)
3,우이신설선,202112,정릉,124880.0,113381.0,238261.0,0.053963,POINT (957048.129 1956079.564)
4,우이신설선,202112,북한산보국문,162606.0,153956.0,316562.0,0.071915,POINT (956599.178 1957073.674)


#### Check CRS

In [9]:
festival_gdf.crs

<Projected CRS: EPSG:5179>
Name: Korea 2000 / Unified CS
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Republic of Korea (South Korea) - onshore and offshore.
- bounds: (122.71, 28.6, 134.28, 40.27)
Coordinate Operation:
- name: Korea Unified Belt
- method: Transverse Mercator
Datum: Geocentric datum of Korea
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [10]:
subway_gdf.crs

<Projected CRS: EPSG:5179>
Name: Korea 2000 / Unified CS
Axis Info [cartesian]:
- X[north]: Northing (metre)
- Y[east]: Easting (metre)
Area of Use:
- name: Republic of Korea (South Korea) - onshore and offshore.
- bounds: (122.71, 28.6, 134.28, 40.27)
Coordinate Operation:
- name: Korea Unified Belt
- method: Transverse Mercator
Datum: Geocentric datum of Korea
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

#### Distance Calculation
* 거리계산을 할때는 `평면직각좌표계(5179, 5186등)`로 변환 후 진행해야 함.

In [11]:
def check_distance_nearest(a_gdf, b_gdf):
    _a_nodes = np.array(list(a_gdf.geometry.apply(lambda node: (node.centroid.x, node.centroid.y))))
    _b_nodes = np.array(list(b_gdf.geometry.apply(lambda node: (node.centroid.x, node.centroid.y))))
    _b_tree = cKDTree(_b_nodes)
    dist, idx = _b_tree.query(_a_nodes, k=1)
    _b_nearest_gdf = b_gdf.iloc[idx].drop(columns='geometry').reset_index(drop=True)
    _gdf = pd.concat([a_gdf.reset_index(drop=True), _b_nearest_gdf, pd.Series(dist, name='dist')], axis=1)
    return _gdf

##### Extracting Nearest Subway for a Festival

In [13]:
CHK_FESTIVAL_NAME = '구로G페스티벌'
special_festival_gdf = festival_gdf[festival_gdf.name == CHK_FESTIVAL_NAME]

In [17]:
nearest_gdf = check_distance_nearest(special_festival_gdf, subway_gdf)

In [18]:
nearest_gdf

,name,organizati,geometry,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,dist
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031),2호선,202112,대림(구로구청),645828.0,660728.0,1306556.0,0.298898,638.881443


##### Extracting Nearest Subway for all Festival

In [19]:
all_nearest_gdf = check_distance_nearest(festival_gdf, subway_gdf)

In [20]:
all_nearest_gdf.head()

,name,organizati,geometry,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,dist
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031),2호선,202112,대림(구로구청),645828.0,660728.0,1306556.0,0.298898,638.881443
1,코로나로 인해 전체 일정 취소,서울특별시 강서구,POINT (942927.290 1952222.192),9호선,202112,가양,544552.0,527052.0,1071604.0,0.245029,691.716009
2,강동선사문화축제,서울특별시 강동구,POINT (967395.183 1951230.727),8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,1108.223457
3,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (965465.452 1949694.336),5호선,202112,광나루(장신대),357309.0,331390.0,688699.0,0.157238,489.585139
4,2022 소원 희망의 빛 거리,서울특별시 광진구,POINT (964472.438 1949878.657),5호선,202112,광나루(장신대),357309.0,331390.0,688699.0,0.157238,567.444225


#### Distance Calculation with GeoPandas

In [21]:
all_nearest_gpd_gdf = gpd.sjoin_nearest(festival_gdf, subway_gdf, distance_col='distances')
all_nearest_gpd_gdf.head()

,name,organizati,geometry,index_right,LINE_NM,YRMN,STATION_NM,IN_CNT,OUT_CNT,IN_OUT,IN_OUT_SCA,distances
0,구로G페스티벌,서울특별시 구로구,POINT (945927.323 1944204.031),342,2호선,202112,대림(구로구청),645828.0,660728.0,1306556.0,0.298898,638.881443
1,코로나로 인해 전체 일정 취소,서울특별시 강서구,POINT (942927.290 1952222.192),53,9호선,202112,가양,544552.0,527052.0,1071604.0,0.245029,691.716009
2,강동선사문화축제,서울특별시 강동구,POINT (967395.183 1951230.727),66,8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,1108.223457
13,제15회 구리한강 유채꽃 축제,경기도 구리시,POINT (967580.935 1952852.498),66,8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,2732.820904
14,제15회 구리 코스모스 축제,경기도 구리시,POINT (967580.935 1952852.498),66,8호선,202112,암사,506279.0,449361.0,955640.0,0.218441,2732.820904


In [22]:
festival_gdf.shape, all_nearest_gpd_gdf.shape

((47, 3), (47, 12))

#### Extracting Radius Distance

##### Specifying a Radius with Buffer

In [24]:
# 700m
special_festival_radius = special_festival_gdf.geometry.buffer(700)
special_festival_radius

0    POLYGON ((946627.323 1944204.031, 946623.952 1...
dtype: geometry

In [32]:
special_festival_radius_gdf = gpd.GeoDataFrame(
    special_festival_gdf[['name']],         # special_festival_gdf.name과 ['name']과 다름
    geometry=special_festival_radius,
    crs='epsg:5179'
)
special_festival_radius_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      1 non-null      object  
 1   geometry  1 non-null      geometry
dtypes: geometry(1), object(1)
memory usage: 24.0+ bytes


In [33]:
special_festival_radius_gdf.explore(
    tiles='http://mt0.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}&s=Ga', 
    attr='google',
)